In [1]:
import os 
from bs4 import BeautifulSoup
#import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from pymongo import MongoClient
from collections.abc import Iterable
import warnings
import time
import requests
import urllib.request as urllib2
from bs4 import BeautifulSoup

In [2]:
#connect to DB
# Global variables
client = MongoClient('localhost', 27017)
db = client['elmenus']
collection = db['elmenus']
# print(cur_path)

selenium_driver_path = "chromedriver"

In [3]:
#convert from selnuim web to text 
def selnuim_to_text(ls):
    lis=[]
    for i in ls:
        lis.append(i.text)
    return lis

In [4]:
#take all P start with •
def clean_req(parh):
    q=parh.text.split('\n')
    ad=[]
    for i in q:
        if "[*•-]" in i:
            ad.append(i)
    return ad


In [5]:
Flag_stop_jobs=4
Flag_id=1
Flag_pages_stop=0

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('/home/omarmohamed/myprojectdir/chromedriver',options=options)
driver.get('https://www.careerbuilder.com/browse')

dict_={}
for j in range(len(driver.find_elements_by_class_name('pb0'))):
    WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'pb0')))
    page_browse= driver.find_elements_by_class_name('pb0')
    industry=page_browse[j].text
    page_browse[j].click()
    WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'link-cta')))
    driver.find_elements_by_class_name("link-cta")[0].click()
    WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'col-2.layout-results')))
    while(len(driver.find_elements_by_class_name('btn.btn-clear.btn-clear-blue.b-i'))):
        try:
            WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME,'btn.btn-clear.btn-clear-blue.b-i')))
            driver.find_elements_by_class_name('btn.btn-clear.btn-clear-blue.b-i')[0].click()
        except:
            break
    time.sleep(2)
    jobs=driver.find_elements_by_class_name("col-2.layout-results")
    print(len(jobs))
    for i in range(4,len(jobs)):
        ul_ls=[]
        WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'col-2.layout-results')))
        driver.find_elements_by_class_name("col-2.layout-results")[i].click()
        time.sleep(3)
        parh=driver.find_elements_by_class_name('col.big.col-mobile-full')[1]
        time.sleep(4)
        ul_ls = parh.find_elements_by_tag_name('ul')
        try:
            print(len(ul_ls))
            if len(ul_ls):
                req=ul_ls[-1]
                Req=req.text.split('\n')
                dict_['requirements']=Req
                


            else:
                Req=clean_req(parh)
                dict_['requirements']=Req
                #Req=parh.text.split('\n')

        except ValueError:
            print("Error in Requirments {\d}".format(Flag_stop))
        print("22")
        recomended_skills=driver.find_elements_by_class_name('check-bubble')
        driver.implicitly_wait(1)
        header = driver.find_elements_by_class_name('data-display-header_content')[0]
        driver.implicitly_wait(1)
        try:
            company_name=header.find_elements_by_tag_name('span')[0].text
        except:
            company_name="not found"
        driver.implicitly_wait(1)
        try:
            location=header.find_elements_by_tag_name('span')[1].text
        except:
            location="not found"
        driver.implicitly_wait(1)
        try:
            job_type=header.find_elements_by_tag_name('span')[2].text
        except:
            job_type="not found"
        
        driver.implicitly_wait(1)
        title=header.find_elements_by_tag_name('h1')[0].text


        try:

            dict_["title"]=title
            dict_['location']=location
            driver.implicitly_wait(1)
            skills=selnuim_to_text(recomended_skills)
            dict_['skills']=skills
            dict_['industry']=industry
            dict_['company_name']=company_name
            dict_['job_type']=job_type
            dict_['_id']=Flag_id

        except Exception as e:
            logf.write("Failed to Scraping {0}:".format(str(Flag_stop)))
        try:
            collection.insert_one(dict_)
        except ValueError:

            logf.write("Failed insert to DB")
        Flag_stop_jobs+=1
        Flag_id+=1
        driver.implicitly_wait(2)
    driver.back()
    driver.back()
    Flag_pages_stop+=1


25
2
22
3
22
